In [116]:
import numpy as np

np.__version__

'1.26.3'

In [117]:
class SNPGenerator:

    def __init__(self, c, r):
        self.inputs = []  # 暂时默认输入到0号
        self.outputs = []  # 暂时不需要输出
        self.cells = np.zeros(c)
        self.rules = np.zeros((c, r, c + 1))

        self.logs = np.empty([0, c], dtype=np.int32)
        self.logs_no_input = np.empty([0, c], dtype=np.int32)

    def load_inputs(self, inputs):
        self.inputs = inputs

    def load_cells(self, cells):
        self.cells = cells

    def load_rules(self, rules):
        self.rules = rules

    def load_default(self):
        self.load_inputs([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])
        self.load_cells(np.array([0, 0, 2, 0]))
        self.load_rules(np.array([[[1, -1, 1, 0, 1],
                                   [0, 0, 0, 0, 0]],

                                  [[1, 0, -1, 1, 0],
                                   [3, 0, -3, 0, 0]],

                                  [[1, 0, 0, -1, 1],
                                   [3, 0, 0, -3, 0]],

                                  [[1, 0, 1, 0, -1],
                                   [2, 0, 1, 0, -2]]]))

    def generate(self):
        # 非空断言
        assert len(self.inputs) > 0, "no input data provided"
        assert self.cells.size > 0, "no cells set"

        for input_spike in self.inputs:
            # 追加无输入日志
            self.logs_no_input = np.vstack((self.logs_no_input, np.array([self.cells])))
            # 输入脉冲信号
            self.cells[0] += input_spike
            # 追加带输入日志
            self.logs = np.vstack((self.logs, np.array(self.cells)))
            
            # 点火计数器和累加器归零
            fired = 0
            update = np.zeros(self.cells.shape[0], dtype=np.int32)
            
            # 遍历整个神经元组，依次计算要点火的规则
            for cid in range(self.cells.shape[0]):
                # 找到目标神经元的内部脉冲数目和规则
                spikes = self.cells[cid]
                rule_mat = self.rules[cid]  # shape=(2,5)
                # 过滤符合阈值的规则         
                cond = np.equal(rule_mat[:, 0], spikes) & np.not_equal(rule_mat[:, 0], 0)
                res = rule_mat[cond]
                # 无规则适合就直接跳过
                if res.shape[0] == 0:
                    continue

                # 
                rng = np.random.default_rng()
                rows = rng.choice(res, 1, replace=False)
                update += rows[0, 1:]
                fired += 1

            # 如果没有规则被执行，且无输入了，那说明系统已经结束了 
            if fired == 0:
                break

            self.cells += update

In [118]:
if __name__ == '__main__':
    # 初始化神经元组
    generator = SNPGenerator(4, 2)
    # 填充数据
    generator.load_default()
    # 运行并记录日志
    generator.generate()
    print(generator.logs)
    # print(generator.logs_no_input)
    # print("end")
    

[[1 0 2 0]
 [0 1 2 1]
 [0 1 3 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 1 0 0]
 [1 0 1 0]
 [1 1 0 2]
 [0 2 1 1]
 [0 3 0 1]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 1 0 0]]
